# Foursquare API Example

## Importing the libraries

In [ ]:
print('Installing dependencies')
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} -c conda-forge geopy
!conda install --yes --prefix {sys.prefix} -c conda-forge folium=0.5.0

In [ ]:
# library to handle requests
import requests 
# library for data analsysis
import pandas as pd 
# library to handle data in a vectorized manner
import numpy as np 
# library for random number generation
import random 

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# plotting library
import folium 

print('Libraries imported')

## Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'WSBDS3PHA2ZA2QRF1K2PFSPE1G2DOMXDFX5LTEJ2NCC5OUG1' # your Foursquare ID
CLIENT_SECRET = 'LOHFOAR0DHZK5WYJOU1N0FRMLVYOUKNYK3KBRCTT33YSEQBH' # your Foursquare Secret
VERSION = '20200403'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Convert an address to latitude and longitude coordinates.

In [ ]:
address = 'Claveria, 02080, Azcapotzalco, Ciudad de Mexico'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Address')
print(address)
print('Coordinates')
print(latitude, longitude)

## Search for a specific venue category
`https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

## Formatting the query

In [ ]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


## Requesting the data

In [ ]:
results = requests.get(url).json()
results

## Getting relevant part of JSON and transform it into a pandas dataframe

In [ ]:
# assign relevant part of JSON to venues
venues_json = results['response']['venues']
# Getting the name of the primary category
for v in venues_json:
    v['categories'] = v['categories'][0]['name']
# tranform venues into a dataframe
venues_df = json_normalize(venues_json)



In [ ]:
venues_df.columns

## Preprocessing 

In [ ]:
venues_clr_df = pd.DataFrame({
    'name': venues_df['name'],
    'categories': venues_df['categories'],
    'address': venues_df['location.address'],
    'distance': venues_df['location.distance'],
    'lat': venues_df['location.lat'],
    'lng': venues_df['location.lng'],
    'city': venues_df['location.city'],
    'state': venues_df['location.state'],
    'country': venues_df['location.country'],
})
venues_clr_df.replace(np.nan, '-', inplace=True)
venues_clr_df.head()

## Generating a map

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(venues_clr_df.lat, venues_clr_df.lng, venues_clr_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map



## Explore Trending Venues (Generalizado)

In [ ]:
# define URL
url_trending = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results_trending = requests.get(url_trending).json()
results_trending



## Getting relevant part of JSON and transform it into a pandas dataframe

In [ ]:
if len(results_trending['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'

else:
    # assign relevant part of JSON to venues
    trending_venues_json = results_trending['response']['venues']
    # Getting the name of the primary category
    for v in trending_venues_json:
        v['categories'] = v['categories'][0]['name']
        print(v)
    # tranform venues into a dataframe
    trending_venues_df = json_normalize(trending_venues_json)
    ## Preprocessing 
    trending_clr_venues_df = pd.DataFrame({
        'name': trending_venues_df['name'],
        'categories': trending_venues_df['categories'],
        'address': trending_venues_df['location.address'],
        'distance': trending_venues_df['location.distance'],
        'lat': trending_venues_df['location.lat'],
        'lng': trending_venues_df['location.lng'],
        'city': trending_venues_df['location.city'],
        'state': trending_venues_df['location.state'],
        'country': trending_venues_df['location.country'],
    })
    trending_clr_venues_df.replace(np.nan, '-', inplace=True)


In [ ]:
trending_clr_venues_df


## Visualize trending venues

In [ ]:
if len(results_trending['response']['venues']) == 0:
    venues_trend_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_trend_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_trend_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_clr_venues_df.lat, trending_clr_venues_df.lng, trending_clr_venues_df.name):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_trend_map)

In [ ]:
venues_trend_map


## Showing venues with popups in a map
`# create map of New York using latitude and longitude values`  
 `map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)`  
 
 `# add markers to map`  
 `for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):`  
     `label = '{}, {}'.format(neighborhood, borough)`  
     `label = folium.Popup(label, parse_html=True)`  
     `folium.CircleMarker(`  
         `[lat, lng],`  
         `radius=5,`  
         `popup=label,`  
         `color='blue',`  
         `fill=True,`  
         `fill_color='#3186cc',`  
         `fill_opacity=0.7,`  
         `parse_html=False).add_to(map_newyork)`  
     
 `map_newyork`  